#DAY 81
House Data from Boston Massachusetts in the 1970s

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split as Train_Test

pd.options.display.float_format = '{:,.2f}'.format #text presentation

data = pd.read_csv('boston.csv', index_col=0) #obs: primeira linha tem os numeros das linhas

In [ ]:
#verificacao dos dados e integridade das infos (NaN e Duplicatas)
#data.shape
#data.columns # column names
#data.sample()
#data.info()
#data.isna().sum()
#data.duplicated().sum()
data.describe()

In [ ]:
#distribuicao de precos

sns.displot(data['PRICE'],
            bins=50,
            aspect=2,
            kde=True,
            color='orange')

plt.title(f'1970s Property Prices in Boston. Average: ${(1000*data.PRICE.mean()):.6}')
plt.xlabel('Price in thousands')
plt.ylabel('Nr. of Homes')

plt.show()

In [ ]:
#distancia das propriedades aos grandes centros
sns.displot(data.DIS,
            bins=50,
            aspect=2,
            kde=True,
            color='lightgreen')

plt.title(f'Distance to Urban Centres. Average: {(data.DIS.mean()):.2} miles')
plt.xlabel('Distance in Miles to 5 Boston Employment Centres')
plt.ylabel('Nr. of Homes')

plt.show()

In [ ]:
#quantidade de quartos por casa
sns.displot(data.RM,
            aspect=2,
            kde=True,
            color='purple')

plt.title(f'Rooms in Boston Houses. Average: {data.RM.mean():.2} rooms')
plt.xlabel('Average Number of Rooms')
plt.ylabel('Nr. of Homes')

plt.show()

In [ ]:
#acesso a rodovias
plt.figure(figsize=(10, 5), dpi=200)

plt.hist(data['RAD'],
         bins=24,
         ec='black',
         color='grey',
         rwidth=0.5)

plt.xlabel('Access to Highways')
plt.ylabel('Nr. of Houses')
plt.show()

In [ ]:
#proximidade a um rio
river_access = data['CHAS'].value_counts()

h_bar = px.bar(x=['No', 'Yes'],
             y=river_access.values,
             color=river_access.values,
             color_continuous_scale=px.colors.sequential.haline,
             title='Proximity to Charles River')

h_bar.update_layout(xaxis_title='Located Next to the River',
                  yaxis_title='Number of Homes',
                  coloraxis_showscale=False)
h_bar.show()

In [ ]:
#graficos da estrutura do dataframe inteiro (Matriz das relações entre tabelas)
sns.pairplot(data)
sns.pairplot(data, kind='reg', plot_kws={'line_kws':{'color': 'red'}}) #regression line
plt.show()

In [ ]:
#JOINTPLOT -> relaçao entre duas variaveis 'DIS' e 'NOX' em um scatterplox
with sns.axes_style('darkgrid'):
  sns.jointplot(x=data['DIS'],
                y=data['NOX'],
                height=8,
                kind='scatter',
                color='deeppink',
                joint_kws={'alpha':0.5})

plt.show()

In [ ]:
#JOINTPLOT -> relaçao entre variaveis 'NOX' e 'INDUS' em um scatterplox
with sns.axes_style('darkgrid'):
  sns.jointplot(x=data.NOX,
                y=data.INDUS,
                # kind='hex',
                height=7,
                color='darkgreen',
                joint_kws={'alpha':0.5})
plt.show()

In [ ]:
#JOINTPLOT -> relaçao entre variaveis 'LSTAT' e 'RM' em um scatterplox
with sns.axes_style('darkgrid'):
  sns.jointplot(x=data['LSTAT'],
                y=data['RM'],
                # kind='hex',
                height=7,
                color='orange',
                joint_kws={'alpha':0.5})
plt.show()

In [ ]:
#JOINTPLOT -> relaçao entre variaveis 'LSTAT' e 'PRICE' em um scatterplox
with sns.axes_style('darkgrid'):
  sns.jointplot(x=data.LSTAT,
                y=data.PRICE,
                # kind='hex',
                height=7,
                color='crimson',
                joint_kws={'alpha':0.5})
plt.show()

In [ ]:
#JOINTPLOT -> 'RM' e 'PRICE' em um scatterplox
with sns.axes_style('whitegrid'):
  sns.jointplot(x=data.RM,
                y=data.PRICE,
                height=7,
                color='darkblue',
                joint_kws={'alpha':0.5})
plt.show()

In [ ]:
#treinamento de modelo preditivo de PREÇO sobre FEATURES(rooms,distance etc)
target = data['PRICE']
features = data.drop('PRICE', axis=1) #restante da DATA

X_train, X_test, y_train, y_test = Train_Test(features,target,test_size=0.2,random_state=10)

# % of training set
train_pct = 100*len(X_train)/len(features)
print(f'Training data is {train_pct:.3}% of the total data.')

# % of test data set
test_pct = 100*X_test.shape[0]/features.shape[0]
print(f'Test data makes up the remaining {test_pct:0.3}%.')

In [ ]:
L_regr = LinearRegression()  #objeto de regressão linear
L_regr.fit(X_train, y_train) #modelo alinhado aos dados de treino (features X_train e alvo y_train)

#ajuste nos dados de treino R² (coeficiente de determinação)
rsquared = L_regr.score(X_train, y_train)
# fração da variância em y explicada pelo modelo
print(f'Training data r-squared: {round(rsquared*100,2)}%')

#extraindo coeficientes de regressão para cada variável
regr_coef = pd.DataFrame(
    data=L_regr.coef_,          #lista de coeficientes aprendidos
    index=X_train.columns,      #nome das colunas correspondentes em X_train
    columns=['Coefficient']     #nome da coluna resultado
)
# Agora regr_coef é um DataFrame com cada variável e seu peso no modelo

#“price premium” por um quarto extra
#'RM' = ROOMS
coef_rm = regr_coef.loc['RM', 'Coefficient']
premium = coef_rm * 1000
print(f'The price premium for having an extra room is ${premium:.5}')
#incremento de preço esperado ao adicionar um cômodo extra

#previsões e resíduos (erros) no conjunto de treino
predicted_values = L_regr.predict(X_train)  #array de valores ŷ estimados pelo modelo
residuals = y_train - predicted_values #diferença entre valores reais e preditos (yᵢ − ŷᵢ)

In [ ]:
#graph superior: Preços Reais vs Preços Preditos
plt.figure(dpi=100)
#plota os pontos reais vs preditos
plt.scatter(
    x=y_train,            #eixo X = valores reais (conjunto de treino)
    y=predicted_values,   #eixo Y = valores preditos pelo modelo
    c='indigo',           #cor dos pontos
    alpha=0.6             #transparencia
)
#linha de referência y = x (concordância real = predito)
plt.plot(
    y_train,  #mesmo vetor em X e Y para formar uma reta
    y_train,
    color='green')
#título e rotulos dos eixos
plt.title(
    'Actual vs Predicted Prices: $y_i$ vs $\\hat{y}_i$',
    fontsize=17)
plt.xlabel(
    'Preços Reais (milhares) $y_i$',
    fontsize=14)
plt.ylabel(
    'Preços Preditos (milhares) $\\hat{y}_i$',
    fontsize=14)
plt.show()

#gragico inferior: Resíduos vs Valores Preditos

plt.figure(dpi=100)
#plota os resíduos em função dos valores preditos
plt.scatter(
    x=predicted_values,  #eixo X: valores preditos pelo modelo
    y=residuals,         #eixo Y: resíduos (y_real - y_predito)
    c='indigo',
    alpha=0.6)
#titulo e rótulos
plt.title(
    'Residuals vs Predicted Values',
    fontsize=17)
plt.xlabel(
    'Preços Preditos $\\hat{y}_i$',
    fontsize=14)
plt.ylabel(
    'Resíduos (erros/ruidos)',
    fontsize=14)
plt.show()

In [ ]:
#stats dos resíduos
resid_mean = round(residuals.mean(), 2)     #media dos residuos rounded
resid_skew = round(residuals.skew(), 2)     #assimetria dos resíduos rounded tbm

#distribuição dos resíduos com KDE (estimativa de densidade)
sns.displot(
    residuals,     #serie de resíduos a ser plotada
    kde=True,      #curva de KDE para visualizar a densidade
    color='red'
)

plt.title(f'Residuals Skew ({resid_skew}) Mean ({resid_mean})')
plt.show()

In [ ]:
#transformando valores para log em razao da regressao para comprimir os valores
plt.figure(dpi=150)
plt.scatter(data.PRICE, np.log(data.PRICE))

plt.title('Original Price to a Log Price')
plt.ylabel('Log Price')
plt.xlabel('Actual $ Price in 000s')
plt.show()

In [ ]:
#Novo target para logarit prices
new_target = np.log(data['PRICE']) # Using log prices
features = data.drop('PRICE', axis=1)
X_train, X_test, log_y_train, log_y_test = Train_Test(features,new_target,test_size=0.2,random_state=10)
log_regr = LinearRegression()
log_regr.fit(X_train, log_y_train)
log_rsquared = log_regr.score(X_train, log_y_train)

log_predictions = log_regr.predict(X_train)
log_residuals = (log_y_train - log_predictions)

print(f'Training data r-squared: {round(log_rsquared*100,2)}%')

In [ ]:
df_coef = pd.DataFrame(data=log_regr.coef_, index=X_train.columns, columns=['coef'])

# Graph of Actual vs. Predicted Log Prices
plt.scatter(x=log_y_train, y=log_predictions, c='navy', alpha=0.6)
plt.plot(log_y_train, log_y_train, color='cyan')
plt.title(f'Actual vs Predicted Log Prices: $y _i$ vs $\hat y_i$ (R-Squared {log_rsquared:.2})', fontsize=17)
plt.xlabel('Actual Log Prices $y _i$', fontsize=14)
plt.ylabel('Prediced Log Prices $\hat y _i$', fontsize=14)
plt.show()

# Original Regression of Actual vs. Predicted Prices
plt.scatter(x=y_train, y=predicted_values, c='indigo', alpha=0.6)
plt.plot(y_train, y_train, color='cyan')
plt.title(f'Original Actual vs Predicted Prices: $y _i$ vs $\hat y_i$ (R-Squared {rsquared:.3})', fontsize=17)
plt.xlabel('Actual prices 000s $y _i$', fontsize=14)
plt.ylabel('Prediced prices 000s $\hat y _i$', fontsize=14)
plt.show()

# Residuals vs Predicted values (Log prices)
plt.scatter(x=log_predictions, y=log_residuals, c='navy', alpha=0.6)
plt.title('Residuals vs Fitted Values for Log Prices', fontsize=17)
plt.xlabel('Predicted Log Prices $\hat y _i$', fontsize=14)
plt.ylabel('Residuals', fontsize=14)
plt.show()

# Residuals vs Predicted values
plt.scatter(x=predicted_values, y=residuals, c='indigo', alpha=0.6)
plt.title('Original Residuals vs Fitted Values', fontsize=17)
plt.xlabel('Predicted Prices $\hat y _i$', fontsize=14)
plt.ylabel('Residuals', fontsize=14)
plt.show()

In [ ]:
#distribuicao de Residuals (log prices) - checking for normality
print(f'Original Model Test Data r-squared: {round(L_regr.score(X_test, y_test)*100,2)}%')
print(f'Log Model Test Data r-squared: {round(log_regr.score(X_test, log_y_test)*100,2)}%')

log_resid_mean = round(log_residuals.mean(), 2)
log_resid_skew = round(log_residuals.skew(), 2)
#com log prices
sns.displot(log_residuals, kde=True, color='navy')
plt.title(f'Log price model: Residuals Skew ({log_resid_skew}) Mean ({log_resid_mean})')
plt.show()
#original
sns.displot(residuals, kde=True, color='indigo')
plt.title(f'Original model: Residuals Skew ({resid_skew}) Mean ({resid_mean})')
plt.show()

In [ ]:
#prediçao de valores a partir da construcao do modelo com LogPrices acima

#starting point: Average Values in the Dataset
features = data.drop(['PRICE'], axis=1)
average_vals = features.mean().values
property_stats = pd.DataFrame(data=average_vals.reshape(1, len(features.columns)),columns=features.columns)

#prediction
log_estimate = log_regr.predict(property_stats)[0]
print(f'The log price estimate is ${log_estimate:.3}')

#convertion of logprices
dollar_est = np.exp(log_estimate) * 1000
print(f'The property is estimated to be worth ${dollar_est:.6}')

In [ ]:
# --- 1. Definição das características da propriedade para previsão customizada ---
next_to_river = False       #proxima ao rio (CHAS)
nr_rooms = 8                #comodos (RM)
students_per_classroom = 20 #indice de alunos
distance_to_town = 5        #Distancia ponderada até o centro da cidade (DIS)

pollution = data.NOX.quantile(q=0.5)    #Mediana de NOX (poluição do ar)
amount_of_poverty = data.LSTAT.quantile(q=0.25)  #(baixa pobreza)

#DataFrame de features no formato que o modelo espera
#property_stats foi inicializado vazio com as mesmas colunas de X_train
property_stats['RM'] = nr_rooms
property_stats['PTRATIO'] = students_per_classroom
property_stats['DIS'] = distance_to_town
property_stats['CHAS'] = 1 if next_to_river else 0
property_stats['NOX'] = pollution
property_stats['LSTAT'] = amount_of_poverty
-
#previsao do log(preço)
log_estimate = log_regr.predict(property_stats)[0]
print(f'The log price estimate is ${log_estimate:.3}')
#converte log-preço dolares
dollar_est = np.exp(log_estimate) * 1000
print(f'The property is estimated to be worth ${dollar_est:.6}')